# Regression - Feature Selection

Feature selection approaches include both discrete and continuous methods. Discrete methods make binary in/out decisions for each feature. They use combinatorial optimization (testing discrete combinations) approaches that quickly become impractical or worse.

Continuous methods apply a penalty to coefficients, reducing the importance of some features relative to another. They use continuous optimization (finding optimal values along a continuous penalty spectrum) approaches, which is much more computationally efficient.

We will cover both, but the class will focus on regularization techniques. They provide a more nuanced way to control model complexity while avoiding the computational challenges and instability of subset selection methods.

## Setup

Load the packages and configure environment.

In [ ]:
%matplotlib inline

import matplotlib.pylab as plt
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

## Discrete Feature Selection - Subset

Discrete feature selection is not supported by SKL. We can use the `mlxtend` library by Sebastian Raschka to demonstrate these methods. It includes `ExhaustiveFeatureSelector`, which implements Best Subset, and `SequentialFeatureSelector`, which implements Forward and Backward Stepwise selection methods. Documentation for these can be found at [the mlxtend homepage](https://rasbt.github.io/mlxtend/).

In [ ]:
# If running on Colab, set up the environment
import sys
if 'google.colab' in sys.modules:
    !pip install mlxtend

### Best Subset with `ExhaustiveFeatureSelector`

Using the Boston data from HW1.

In [ ]:
# download the data set directly from the web using pandas
url = "https://raw.githubusercontent.com/olearydj/INSY7120/refs/heads/main/notebooks/data/Boston.csv"
boston = pd.read_csv(url)

In [ ]:
boston.head()

Legend from HW1:

- crim: per capita crime rate by town.
- zn: proportion of residential land zoned for lots over 25,000 sq.ft.
- indus: proportion of non-retail business acres per town.
- chas: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
- nox: nitrogen oxides concentration (parts per 10 million).
- rm: average number of rooms per dwelling.
- age: proportion of owner-occupied units built prior to 1940.
- dis: weighted mean of distances to five Boston employment centres.
- rad: index of accessibility to radial highways.
- tax: full-value property-tax rate per \$10,000.
- ptratio: pupil-teacher ratio by town.
- lstat: lower status of the population (percent).
- medv: median value of owner-occupied homes in \$1000s.

There are 12 predictors and one outcome, `crim`.

In [ ]:
# get the predictors of interest
X = boston.loc[:,'zn':]
y = boston[['crim']]

In [ ]:
X.head()

Then use `ExhaustiveFeatureSelection` from mlxtend to specify the range of feature sets you want to explore.

From [the documentation](https://arc.net/l/quote/xdfbqtco) (or `help(EFS)` after import), EFS takes a number of parameters:

```text
ExhaustiveFeatureSelector(estimator, min_features=1, max_features=1, print_progress=True, scoring='accuracy', cv=5, n_jobs=1, pre_dispatch='2*n_jobs', clone_estimator=True, fixed_features=None, feature_groups=None)
```

Of most immediate interest to us are:

- `estimator`: the SKL classifier or regressor
- `min_features`: the minimum number of features to select
- `max_features`: the maximum number of features to select
- `scoring`: the scoring metric used
- `print_progress`: prints progress (default: True)

The `scoring` parameter documentation here seems out of date. I believe that it can use any metric that is appropriate for regression. We have discussed RSE, MSE, RMSE, and R^2. To get a list of the metrics available in SKL: 

In [ ]:
from sklearn.metrics import get_scorer_names

get_scorer_names()

Of these, `neg_mean_squared_error` (MSE), `neg_root_mean_squared_error` (RMSE), and `r2` match. RSE is not included. Note that all these values are negated (as indicated by the `neg_` prefix). This is to match SKL's convention that higher scoring values are better. Recall that low values of MSE and RMSE are good, so negating them ensures that higher values represent better models.

To use EFS we create an instance of it with the desired parameters, including the model. Then we fit the resulting "meta-model" using `r2`.

In [ ]:
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS

lr = LinearRegression()
# turn off cross-validation and progress outputs
efs = EFS(lr, min_features=5, max_features=12, scoring='r2', cv=0, print_progress=False)

Testing all combinations of 5 to 12 features. This will take a sec. For $p$ predictors,

$$\text{models built} = \sum_{k=min}^{max} \binom{p}{k} = \sum_{k=min}^{max} \frac{p!}{k!(p-k)!}$$

In our case, $p = 12$, $min = 5$, and $max = 12$, so the number of models is 3,302. Here is a simple function to calculate that.

In [ ]:
def count_subset_models(total_predictors, min_features, max_features):
    from math import comb
    
    # Ensure max_features doesn't exceed total predictors
    max_features = min(max_features, total_predictors)
    
    # Sum up combinations for each feature count
    total_models = 0
    for k in range(min_features, max_features + 1):
        total_models += comb(total_predictors, k)
    
    return total_models

print(count_subset_models(12, 5, 12))

We can use the `%timeit` command in Jupyter Lab to time the execution. Normally this runs the specified command `r` times for each `n` loops so that mean and standard deviations can be reported. Here the `-r1` and `-n1` arguments tell it to perform the fit only once (one run, one loop).

In [ ]:
%timeit -r1 -n1 efs.fit(X, y)

The scores and subset are stored in the `best_score_` and `best_idx_` attributes of the fitted model. `best_feature_names_` holds the corresponding column names.

In [ ]:
print(f"Best R^2: {efs.best_score_:.2f}")
print(f"Best Subset: {efs.best_idx_}")
print(efs.best_feature_names_)

Note: this process selected all the predictors. Why?

The `subsets_` attribute holds all the combinations tested and their scores. It is a dictionary with integer index:

In [ ]:
efs.subsets_[0]

We can use its length to confirm that the expected number of models were fit.

In [ ]:
len(efs.subsets_)

Now that we have the "best" features, we have to build and fit a new model, and evaluate the result.

First, use the EFS method to transform the feature set. Then fit, etc. as usual.

In [ ]:
# this is equivalent to X_best = X[:, efs.best_idx_]
X_best = efs.transform(X)

lr_best = LinearRegression()
lr_best.fit(X_best, y)

In [ ]:
# look at the estimated model parameters
print(f"Model Coefficients: {lr_best.coef_}")
print(f"Model Intercept: {lr_best.intercept_}")

r2 = lr_best.score(X_best,y)
print(f"R² Score: {r2:.2f}")

### Forward and Backward Stepwise with `SequentialFeatureSelector`

The `SequentialFeatureSelector` class implements both forward and backwards stepwise subset selection. It's interface (described at [this link](https://rasbt.github.io/mlxtend/user_guide/feature_selection/SequentialFeatureSelector/)) is similar to that used by EFS. There are differences in the parameters and attributes.

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

lr = LinearRegression()
# turn off cross-validation and progress outputs
forward = SFS(lr, k_features=5, forward=True, scoring='r2', cv=0)

In [ ]:
%timeit -r1 -n1 forward.fit(X, y)

In [ ]:
print(f"R^2: {forward.k_score_:.2f}")
print(f"Best Subset: {forward.k_feature_idx_}")
print(forward.k_feature_names_)

In [ ]:
forward.subsets_

Compare with backwards:

In [ ]:
backward = SFS(lr, k_features=5, forward=False, scoring='r2', cv=0)
backward.fit(X, y)
print(f"R^2: {backward.k_score_:.2f}")
print(f"Best Subset: {backward.k_feature_idx_}")
print(backward.k_feature_names_)
backward.subsets_

## Continuous Feature Selection - Regularization (aka Shrinkage)

### Ridge Regression

Ridge Regression and Lasso are sensitive to predictors with different value scales. Need to standardize data first.

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Create a ridge regression model with regularization strength alpha=1.0
ridge_model = Ridge(alpha=1.0)

# Fit the model
ridge_model.fit(X_scaled, y)

In [ ]:
print(f"Coefficients: {ridge_model.coef_}")
print(f"Intercept: {ridge_model.intercept_}")

In [ ]:
# Get R² score on training data
r2 = ridge_model.score(X_scaled, y)
print(f"Ridge R² on training data: {r2:.4f}")

### The Lasso

In [ ]:
from sklearn.linear_model import Lasso

# Create a ridge regression model with regularization strength alpha=1.0
lasso_model = Lasso(alpha=1.0)

# Fit the model
lasso_model.fit(X_scaled, y)

In [ ]:
print(f"Coefficients: {lasso_model.coef_}")
print(f"Intercept: {lasso_model.intercept_}")

In [ ]:
# Get R² score on training data
r2 = lasso_model.score(X_scaled, y)
print(f"Lasso R² on training data: {r2:.4f}")

In [ ]:
feature_names = X.columns
non_zero_features = feature_names[lasso_model.coef_ != 0]
print(non_zero_features)

## Plots for Slides

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define functions for variance and bias²
def variance(lambda_val):
    return 1 / (1 + lambda_val)

def bias_squared(lambda_val):
    return (lambda_val**2) / ((1 + lambda_val)**2)

def mse(lambda_val):
    # MSE = Variance + Bias² (ignoring irreducible error)
    return variance(lambda_val) + bias_squared(lambda_val)

# Create lambda values - focusing on 0 to 4 range
lambda_vals = np.linspace(0, 4, 1000)

# Calculate values
var_vals = variance(lambda_vals)
bias_sq_vals = bias_squared(lambda_vals)
mse_vals = mse(lambda_vals)

# Create the plot
plt.figure(figsize=(10, 6))

# Plot the curves
plt.plot(lambda_vals, var_vals, 'g-', linewidth=2.5, label='Variance ∝ 1/(1+λ)')
plt.plot(lambda_vals, bias_sq_vals, 'k-', linewidth=2.5, label='Bias² ∝ λ²/(1+λ)²')
plt.plot(lambda_vals, mse_vals, 'purple', linewidth=2.5, label='MSE = Variance + Bias²')

# Find and mark the minimum MSE point
min_mse_idx = np.argmin(mse_vals)
min_mse_lambda = lambda_vals[min_mse_idx]
min_mse_value = mse_vals[min_mse_idx]

plt.plot(min_mse_lambda, min_mse_value, 'ro', markersize=8)
plt.annotate(f'Optimal λ ≈ {min_mse_lambda:.2f}', 
             xy=(min_mse_lambda, min_mse_value),
             xytext=(min_mse_lambda+0.5, min_mse_value+0.05),
             arrowprops=dict(facecolor='red', shrink=0.05),
             fontsize=12)

# Set up the plot
plt.xlabel('λ (Lambda)', fontsize=14)
plt.ylabel('Value', fontsize=14)
plt.title('Variance-Bias² Tradeoff in Ridge Regression', fontsize=16, fontweight='bold')
plt.legend(loc='best', fontsize=12)
plt.grid(True, alpha=0.3)

# Add annotations to explain key regions - adjusted for zoomed view
plt.annotate('Variance decreases rapidly', 
             xy=(0.3, variance(0.3)), 
             xytext=(0.5, 0.6),
             arrowprops=dict(facecolor='green', shrink=0.05),
             fontsize=12)

plt.annotate('Bias² increases slowly at first', 
             xy=(0.5, bias_squared(0.5)), 
             xytext=(1.5, 0.1),
             arrowprops=dict(facecolor='black', shrink=0.05),
             fontsize=12)

plt.annotate('MSE improves', 
             xy=(0.5, mse(0.5)), 
             xytext=(1.0, 0.8),
             arrowprops=dict(facecolor='purple', shrink=0.05),
             fontsize=12)

# Mark the OLS point (λ=0)
plt.plot(0, variance(0), 'bo', markersize=8)
plt.annotate('OLS (λ=0)', 
             xy=(0, variance(0)),
             xytext=(0.2, 0.9),
             arrowprops=dict(facecolor='blue', shrink=0.05),
             fontsize=12)

# Zoom in on the 0-4 lambda range as requested
plt.xlim(-0.1, 4)
plt.ylim(-0.05, 1.05)

# Add a clear marker at λ=0 showing variance=1, bias²=0
plt.axhline(y=1.0, color='gray', linestyle='--', alpha=0.3)
plt.axhline(y=0.0, color='gray', linestyle='--', alpha=0.3)

# Show the plot
plt.tight_layout()
plt.show()

# For the second plot - rates of change
plt.figure(figsize=(10, 6))

# Small delta for numerical differentiation
delta = 0.0001
lambda_vals_dense = np.linspace(0.0001, 4, 1000)  # Avoid division by zero

# Compute numerical derivatives
var_derivative = (variance(lambda_vals_dense + delta) - variance(lambda_vals_dense)) / delta
bias_derivative = (bias_squared(lambda_vals_dense + delta) - bias_squared(lambda_vals_dense)) / delta

# Plot the derivatives
plt.plot(lambda_vals_dense, -var_derivative, 'g-', linewidth=2.5, 
         label='Rate of variance decrease')
plt.plot(lambda_vals_dense, bias_derivative, 'k-', linewidth=2.5, 
         label='Rate of bias² increase')

# Intersection point where rates are equal
intersection_idx = np.argmin(np.abs(-var_derivative - bias_derivative))
intersection_lambda = lambda_vals_dense[intersection_idx]

plt.plot(intersection_lambda, -var_derivative[intersection_idx], 'ro', markersize=8)
plt.annotate('Equal rates\npoint', 
             xy=(intersection_lambda, -var_derivative[intersection_idx]),
             xytext=(intersection_lambda+0.5, -var_derivative[intersection_idx]+0.1),
             arrowprops=dict(facecolor='red', shrink=0.05),
             fontsize=12)

# Set up the plot
plt.xlabel('λ (Lambda)', fontsize=14)
plt.ylabel('Rate of change', fontsize=14)
plt.title('Rates of Change for Variance and Bias² with λ', fontsize=16, fontweight='bold')
plt.legend(loc='best', fontsize=12)
plt.grid(True, alpha=0.3)

# Add annotations
plt.annotate('Variance changes rapidly at small λ', 
             xy=(0.1, -var_derivative[10]), 
             xytext=(1.5, 0.8),
             arrowprops=dict(facecolor='green', shrink=0.05),
             fontsize=12)

plt.annotate('Bias² changes slowly at small λ', 
             xy=(0.1, bias_derivative[10]), 
             xytext=(2, 0.2),
             arrowprops=dict(facecolor='black', shrink=0.05),
             fontsize=12)

# Zoom in on the 0-4 lambda range
plt.xlim(-0.1, 4)
plt.ylim(-0.05, 1.05)

plt.tight_layout()
plt.show()